In [1]:
!pip install faiss-cpu

     ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
     --------------------------------------- 0.0/10.8 MB 262.6 kB/s eta 0:00:41
     --------------------------------------- 0.1/10.8 MB 491.5 kB/s eta 0:00:22
     - -------------------------------------- 0.5/10.8 MB 2.3 MB/s eta 0:00:05
     ----- ---------------------------------- 1.4/10.8 MB 6.5 MB/s eta 0:00:02
     ---------- ----------------------------- 2.8/10.8 MB 10.6 MB/s eta 0:00:01
     ------------- -------------------------- 3.7/10.8 MB 11.9 MB/s eta 0:00:01
     ----------------- ---------------------- 4.8/10.8 MB 13.3 MB/s eta 0:00:01
     --------------------- ------------------ 5.9/10.8 MB 15.0 MB/s eta 0:00:01
     -------------------------- ------------- 7.2/10.8 MB 15.8 MB/s eta 0:00:01
     ------------------------------- -------- 8.5/10.8 MB 17.6 MB/s eta 0:00:01
     ------------------------------------ --- 9.8/10.8 MB 

In [2]:
!pip install sentence-transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/f9/e6/3c821e7417acd82df89e39f09156ce80d58817b5b4b1ac5453b522bc5dd4/torchvision-0.16.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/977.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/977.5 kB ? eta -:--:--
     - ----------------------------------- 41.0/977.5 kB 487.6 kB/s eta 0:00:02
     ------- ------------------------------ 204.8/977.5 kB 1.8 MB/s eta 0:00:01
     -------------------------------------  972.8/977.5 kB 6.8 MB/s eta 0:00:01
     -------------------------------------- 977.5/977.5 kB 5.6 MB/s eta 0:00:00
  Obtaining dependency information for torch>=1.6.0 from https://files.pythonhosted.org/packages/e4/ae/2ad8820045b6631965750435f28583e80905b8273d57cf026163b51323ee/torch-2.1.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0

In [4]:
#Loading a sample dataset with 5 reviews
import pandas as pd
import numpy as np
df=pd.read_csv('Reviews.csv')
df=df.head(5)
df.head()

,Review,Rating
0,This dress is perfection! so pretty and flatte...,3.0
1,"A flattering, super cozy coat. will work well...",4.0
2,"If this product was in petite, i would get the...",5.0
3,This is a nice choice for holiday gatherings. ...,5.0
4,I love the look and feel of this tulle dress.,5.0


In [6]:
from sentence_transformers import SentenceTransformer
text = df['Review']
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
vectors = encoder.encode(text)

In [7]:
vectors

array([[-0.06217552,  0.07378877, -0.07448871, ...,  0.07027042,
         0.01796084, -0.09390768],
       [-0.12158108, -0.15971243,  0.0704374 , ...,  0.10647692,
        -0.06763223,  0.03161925],
       [ 0.24184246, -0.10951386, -0.04839725, ...,  0.09273516,
        -0.14525726, -0.08345813],
       [-0.11661569, -0.18936978, -0.01969846, ...,  0.22593035,
         0.05131904, -0.03293313],
       [-0.02752496,  0.03535304, -0.04940617, ...,  0.15104961,
         0.12293017, -0.02794872]], dtype=float32)

In [9]:
len(vectors[0])

768

In [11]:
import faiss

vector_dimensions = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimensions)
faiss.normalize_L2(vectors)
index.add(vectors)

In [12]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000225B74992F0> >

In [13]:
search_text = 'How good is the quality of the dress'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

In [14]:
_vector

array([[ 2.07733717e-02,  7.57977813e-02,  9.20572598e-03,
         3.13423686e-02, -3.74763831e-02, -2.62014680e-02,
        -1.56621411e-02,  1.28351431e-02, -3.46761458e-02,
        -2.34523732e-02,  1.47250423e-03, -2.84027285e-03,
         3.27750705e-02,  2.57291924e-02, -9.29515343e-03,
        -2.89442148e-02, -1.74799804e-02,  5.08571491e-02,
        -2.53807846e-02,  2.91293431e-02,  2.63922266e-04,
        -2.63975207e-02,  1.06971944e-02, -3.09604919e-03,
        -5.32037392e-03, -1.54564651e-02,  2.20694835e-03,
        -1.02199498e-03, -3.45763527e-02,  4.02387008e-02,
         5.11127263e-02, -3.12978029e-02, -1.25781428e-02,
        -6.43045679e-02,  1.70842577e-02, -2.97773164e-02,
         2.19105631e-02, -5.20133823e-02,  8.44690669e-03,
         4.14445512e-02,  2.90970523e-02,  2.59141419e-02,
        -1.73868798e-02, -1.21797090e-02,  2.70887371e-02,
        -3.54617313e-02, -1.27860513e-02,  9.06199291e-02,
        -3.17766070e-02,  5.73248137e-03, -1.59371309e-0

In [19]:
k = index.ntotal
distances, src = index.search(_vector, k=k)

In [16]:
k

5

In [20]:
result = pd.DataFrame({'distances': distances[0], 'src': src[0]})

In [21]:
result

,distances,src
0,0.773289,0
1,0.833064,4
2,1.132186,2
3,1.142530,3
4,1.314004,1


In [24]:
join=pd.merge(result,df,left_on='src',right_index=True)

In [25]:
join

,distances,src,Review,Rating
0,0.773289,0,This dress is perfection! so pretty and flatte...,3.0
1,0.833064,4,I love the look and feel of this tulle dress.,5.0
2,1.132186,2,"If this product was in petite, i would get the...",5.0
3,1.142530,3,This is a nice choice for holiday gatherings. ...,5.0
4,1.314004,1,"A flattering, super cozy coat. will work well...",4.0


In [26]:
df['Review'][0]

'This dress is perfection! so pretty and flattering.'